XSSClassifier:  XSS Attack Detection Approach Based on Machine Learning Classifier in Python

## 1. Gather Test Data 
Get a list of samples and create an array with labels.

In [1]:
import sys
!{sys.executable} -m pip install -U numpy
!{sys.executable} -m pip install -U gensim
!{sys.executable} -m pip install -U python-Levenshtein
!{sys.executable} -m pip install -U nltk
!{sys.executable} -m pip install -U scikit-learn

# Importing necessary libraries
import warnings
#warnings.filterwarnings("ignore")
import nltk
nltk.download('punkt')

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from numpy import *
from urllib.parse import unquote

import numpy as np
import pandas as pd
import csv
import urllib.parse as url_parse
import pickle

# Initialize lists for XSS and normal data
xss_data = []
normal_data = []
X_temp = []
X = []
y = []
xss_count = 0
normal_count = 0

print("Fetching Data...")

# Load XSS strings and label them as 1 in the y array
with open('lib/xss_samples.txt', 'r') as f:
    xss_data = f.readlines()
print("*", sep=' ', end='', flush=True)

# Extract the query part of the URL and filter out potential open redirect vulnerabilities
for line in xss_data:
    query = url_parse.urlsplit(line)[3]
    if "?http" in str(line) or "?url=http" in str(line) or "?fwd=http" in str(line) or "?path=http" in str(line) or "=http" in str(query) or "page=search" in str(query):
        continue
    if len(query) > 8:
        xss_count += 1
        X_temp.append(line)

# Remove duplicates
deduplicated_xss = list(dict.fromkeys(X_temp))
print("*", sep=' ', end='', flush=True)

# Add features to X and labels to the y array for XSS data
for line in deduplicated_xss:
    X.append(line)
    y.append(1)

X_temp = []
deduplicated_xss = []
print("*", sep=' ', end='', flush=True)

# Load normal strings and label them as 0 in the y array
with open('lib/normal_samples.txt', 'r', encoding='utf-8', errors='ignore') as f:
    normal_data = f.readlines()

# Extract the query part of the URL
for line in normal_data:
    query = url_parse.urlsplit(line)[3]
    if len(query) > 3:
        normal_count += 1
        X_temp.append(line)

# Remove duplicates
deduplicated_normal = list(dict.fromkeys(X_temp))
print("*", sep=' ', end='', flush=True)

# Add features to X and labels to the y array for normal data
for line in deduplicated_normal:
    X.append(line)
    y.append(0)



[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: C:\Users\hkrdh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: C:\Users\hkrdh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: C:\Users\hkrdh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: C:\Users\hkrdh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: C:\Users\hkrdh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hkrdh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Fetching Data...
****

In [4]:
print("Number of XSS Samples: "+str(xss_count))
print("Number of NOT XSS Samples: "+str(normal_count))
print("Total Samples: "+str(xss_count+normal_count))

Number of XSS Samples: 38608
Number of NOT XSS Samples: 44826
Total Samples: 83434


## 2. Create an array of the features for each test sample

In [6]:
# Create a function to convert an array of query strings to a set of features
def generateFeatures(text_data):
    tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(text_data)]
    max_epochs = 25
    vec_size = 20
    alpha = 0.025

    model = Doc2Vec(vector_size=vec_size,
                    alpha=alpha,
                    min_alpha=0.00025,
                    min_count=1,
                    dm=1)
    model.build_vocab(tagged_data)
    print("Building the sample vector model...")
    features = []

    for epoch in range(max_epochs):
        print("*", sep=' ', end='', flush=True)
        model.random.seed(42)
        model.train(tagged_data,
                    total_examples=model.corpus_count,
                    epochs=model.epochs)
        model.alpha -= 0.0002
        model.min_alpha = model.alpha

    model.save("lib/d2v_model")
    print()
    print("Model Saved")

    for i, line in enumerate(text_data):
        feature_vec = [model.dv[i]]
        line_decoded = unquote(line)
        line_decoded = line_decoded.replace(" ", "")
        lower_str = str(line_decoded).lower()

        # Feature extraction
        feature1 = int(lower_str.count('<link')) + int(lower_str.count('<object')) + int(lower_str.count('<form'))
        # Add more features here...

        feature_vec = np.append(feature_vec, feature1)
        # Add more features here...

        features.append(feature_vec)

    return features

# Example usage
X_features = generateFeatures(X)


Building the sample vector model...
*******

KeyboardInterrupt: 

In [8]:
features = generateFeatures(X)
features_dict = {'data':X,'features':features,'label':y}

### Features Data

In [ ]:
print("Test Sample: "+ X[0])
print("Features: " + str(features[0]))
print("\nLabel:\033[1;31;1m XSS(1)/\033[1;32;1m NOT XSS(0)\033[0;0m: " + str(y[0]))

Test Sample: http://search.rin.ru/cgi-bin/find.cgi?text=%3Cscript%3Ealert(%27HZ+iz+1337%27)%3B%3C%2Fscript%3E

Features: [ 1.70212924 -1.82771647  2.17515659  2.02932262 -0.75966585 -1.18975067
 -0.71887583  0.25190741  1.54317236 -1.36383951  2.34889245  0.26069176
  2.36285257 -0.36768007  1.09289861  1.08891952 -1.69099092  3.48596787
 -1.11877799 -1.52294385  0.          0.          0.         81.
  1.         15.        ]

Label: XSS(1)/ NOT XSS(0): 1


## 3. Train the Model

Train/Test split the training data

In [11]:
np.random.seed(42)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size = .3, random_state=42)


In [12]:
# Use RandomState for reproducibility.
from sklearn import tree
my_classifier1 = tree.DecisionTreeClassifier(random_state=42)
print(my_classifier1)
print()

from sklearn.svm import SVC
my_classifier2 = SVC(kernel='linear', random_state=42)
print(my_classifier2)
print()

from sklearn.naive_bayes import GaussianNB
my_classifier3 = GaussianNB()
print(my_classifier3)
print()

from sklearn.neighbors import KNeighborsClassifier
my_classifier4 = KNeighborsClassifier(n_neighbors=25, weights='uniform')
print(my_classifier4)
print()

from sklearn.ensemble import RandomForestClassifier
my_classifier5 = RandomForestClassifier(random_state=42)
print(my_classifier5)
print()

from sklearn.neural_network import MLPClassifier
my_classifier6 = MLPClassifier(max_iter=2000, random_state=42)
print(my_classifier6)
print()

DecisionTreeClassifier(random_state=42)

SVC(kernel='linear', random_state=42)

GaussianNB()

KNeighborsClassifier(n_neighbors=25)

RandomForestClassifier(random_state=42)

MLPClassifier(max_iter=2000, random_state=42)



#### Additonal tunning of hyperparameters can be done.

I have only set the KNN n_neighbors weights

In [13]:
print("Training Classifier #1 DecisionTreeClassifier")
my_classifier1.fit(X_train, y_train)
print("Training Classifier #2 SVC")
my_classifier2.fit(X_train, y_train)
print("Training Classifier #3 GaussianNB")
my_classifier3.fit(X_train, y_train)
print("Training Classifier #4 KNeighborsClassifier")
my_classifier4.fit(X_train, y_train)
print("Training Classifier #5 RandomForestClassifier")
my_classifier5.fit(X_train, y_train)
print("Training Classifier #6 MLPClassifier")
my_classifier6.fit(X_train, y_train)

predictions1 = my_classifier1.predict(X_test)
predictions2 = my_classifier2.predict(X_test)
predictions3 = my_classifier3.predict(X_test)
predictions4 = my_classifier4.predict(X_test)
predictions5 = my_classifier5.predict(X_test)
predictions6 = my_classifier6.predict(X_test)


Training Classifier #1 DecisionTreeClassifier
Training Classifier #2 SVC
Training Classifier #3 GaussianNB
Training Classifier #4 KNeighborsClassifier
Training Classifier #5 RandomForestClassifier
Training Classifier #6 MLPClassifier


### Training Accuracy Score

In [14]:
from sklearn.metrics import accuracy_score
print('Accuracy Score #1: {:.1%}'.format(accuracy_score(y_test, predictions1)))
print('Accuracy Score #2: {:.1%}'.format(accuracy_score(y_test, predictions2)))
print('Accuracy Score #3: {:.1%}'.format(accuracy_score(y_test, predictions3)))
print('Accuracy Score #4: {:.1%}'.format(accuracy_score(y_test, predictions4)))
print('Accuracy Score #5: {:.1%}'.format(accuracy_score(y_test, predictions5)))
print('Accuracy Score #6: {:.1%}'.format(accuracy_score(y_test, predictions6)))

Accuracy Score #1: 98.5%
Accuracy Score #2: 98.4%
Accuracy Score #3: 94.9%
Accuracy Score #4: 96.2%
Accuracy Score #5: 99.5%
Accuracy Score #6: 99.6%


### Classification Report

In [15]:
from sklearn.metrics import classification_report
print("Classification Report #1 DecisionTreeClassifier")
print(classification_report(y_test, predictions1))
print("Classification Report #2 SVC")
print(classification_report(y_test, predictions2))
print("Classification Report #3 GaussianNB")
print(classification_report(y_test, predictions3))
print("Classification Report #4 KNeighborsClassifier")
print(classification_report(y_test, predictions4))
print("Classification Report #5 RandomForestClassifier")
print(classification_report(y_test, predictions5))
print("Classification Report #6 MLPClassifier")
print(classification_report(y_test, predictions6))

Classification Report #1 DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     13437
           1       0.98      0.98      0.98     11594

    accuracy                           0.98     25031
   macro avg       0.98      0.98      0.98     25031
weighted avg       0.98      0.98      0.98     25031

Classification Report #2 SVC
              precision    recall  f1-score   support

           0       0.98      0.99      0.99     13437
           1       0.99      0.97      0.98     11594

    accuracy                           0.98     25031
   macro avg       0.98      0.98      0.98     25031
weighted avg       0.98      0.98      0.98     25031

Classification Report #3 GaussianNB
              precision    recall  f1-score   support

           0       0.96      0.94      0.95     13437
           1       0.94      0.95      0.95     11594

    accuracy                           0.95     25031
   macro avg   

In [16]:
from sklearn.metrics import confusion_matrix
print("\nConfusion Matrix #1 DecisionTreeClassifier")
print(confusion_matrix(y_test, predictions1))
print("\nConfusion Matrix #2 SVC")
print(confusion_matrix(y_test, predictions2))
print("\nConfusion Matrix #3 GaussianNB")
print(confusion_matrix(y_test, predictions3))
print("\nConfusion Matrix #4 KNeighborsClassifier")
print(confusion_matrix(y_test, predictions4))
print("\nConfusion Matrix #5 RandomForestClassifier")
print(confusion_matrix(y_test, predictions5))
print("\nConfusion Matrix #6 MLPClassifier")
print(confusion_matrix(y_test, predictions6))


Confusion Matrix #1 DecisionTreeClassifier
[[13249   188]
 [  192 11402]]

Confusion Matrix #2 SVC
[[13344    93]
 [  305 11289]]

Confusion Matrix #3 GaussianNB
[[12690   747]
 [  533 11061]]

Confusion Matrix #4 KNeighborsClassifier
[[13269   168]
 [  780 10814]]

Confusion Matrix #5 RandomForestClassifier
[[13419    18]
 [  110 11484]]

Confusion Matrix #6 MLPClassifier
[[13411    26]
 [   86 11508]]


In [17]:
print("Training Classifier #1 DecisionTreeClassifier")
my_classifier1.fit(features, y)

print("Training Classifier #2 SVC")
my_classifier2.fit(features, y)

print("Training Classifier #3 GaussianNB")
my_classifier3.fit(features, y)

print("Training Classifier #4 KNeighborsClassifier")
my_classifier4.fit(features, y)

print("Training Classifier #5 RandomForestClassifier")
my_classifier5.fit(features, y)

print("Training Classifier #6 MLPClassifier")
my_classifier6.fit(features, y)

Training Classifier #1 DecisionTreeClassifier
Training Classifier #2 SVC
Training Classifier #3 GaussianNB
Training Classifier #4 KNeighborsClassifier
Training Classifier #5 RandomForestClassifier
Training Classifier #6 MLPClassifier


MLPClassifier(max_iter=2000, random_state=42)

In [18]:
# save the model to disk
filename1 = 'lib/DecisionTreeClassifier.sav'
pickle.dump(my_classifier1, open(filename1, 'wb'))

filename2 = 'lib/SVC.sav'
pickle.dump(my_classifier2, open(filename2, 'wb'))

filename3 = 'lib/GaussianNB.sav'
pickle.dump(my_classifier3, open(filename3, 'wb'))

filename4 = 'lib/KNeighborsClassifier.sav'
pickle.dump(my_classifier4, open(filename4, 'wb'))

filename5 = 'lib/RandomForestClassifier.sav'
pickle.dump(my_classifier5, open(filename5, 'wb'))

filename6 = 'lib/MLPClassifier.sav'
pickle.dump(my_classifier6, open(filename6, 'wb'))

In [19]:
# load the model from disk
loaded_model1 = pickle.load(open(filename1, 'rb'))
loaded_model2 = pickle.load(open(filename2, 'rb'))
loaded_model3 = pickle.load(open(filename3, 'rb'))
loaded_model4 = pickle.load(open(filename4, 'rb'))
loaded_model5 = pickle.load(open(filename5, 'rb'))
loaded_model6 = pickle.load(open(filename6, 'rb'))

In [23]:
testXSS = [
                '<script>alert(\'xss\')</script><script><script>',
                'hellomo',
                'https://store.bentley.com/en/shop/search?term=%22%3E%3Cdetails%20open%20ontoggle=prompt(1337)%3ExxLouisLouisLouis',
                'ghfdhgdhjgd',
                'uid%3D19%26list_page%3D%22%3E%3Cscript%3Ealert%28document.cookie%29%3B%3C/script%3E',
                '&template=en_search_error&postalCode=\\\';alert(0)//',
                '&where=%3Cscript%3Ealert%28%27xss%27%29%3C%2Fscript%3E&loctypes=1003%2C1001%2C1000%2C1%2C9%2C5%2C11%2C13%2C19%2C20&from=hdr_localsearch',
                'http://mydata.com/sad/sd/qwd/qwde/qwe/?sessionid=12',
                'http://mydata.com?id=script',
                '&\';}},{scope:\'email,user_about_me,user_hometown,user_interests,user_likes,user_status,user_website,user_birthday,publish_stream,publish_actions,offline_access\'});}alert(0);b=function(response){c=({a:{//',
                'http://myurl.com?<script',
                'http://mydata.com?script=script',
                'composite_search=1&keyword="/><script>alert("Xss:Vijayendra")</script>',
                'http://mysite.com?srtalert',
                'script',
                'alert',
                'Search=%22%3E\'%3E%3CSCRIPT%20SRC=http://br.zone-h.org/testes/xss.js%3E%3C/SCRIPT%3E?',
                'id=15%3Cscript%3Ealert%28document.cookie%29%3C/script%3E',
                'composite_search=1&keyword="/><script>alert("Xss:Vijayendra")</script>',
                'id=123&href=abdc<a<script>alert(1)',
                '<<<<<<>>>>></>,><><>',
                'alert()alert()',
                'alertalert',
                '?url=http://localhost:8888/notebooks/Documents/MachineLearning/Practical%20Machine%20Learning',
                '<script<script',
                '<scriptalert',
                'httphttphttp',
                'https://disqus.com/?ref_noscript',
                'I am a string',
                '<img src="javascript:alert(1)/>"',
                'HelloWorld!',
                'http://mysite.com?<script>',
                '<input type="text" value=`` <div/onmouseover=\'alert(471)\'>X</div>',
                '<img \x47src=x onerror="javascript:alert(324)">',
                '<a href="\xE2\x80\x87javascript:javascript:alert(183)" id="fuzzelement1">test</a>',
                '<body onscroll=javascript:alert(288)><br><br><br><br><br><br>...<br><br><br><br><br><br><br><br><br><br>...<br><br><br><br><br><br><br><br><br><br>...<br><br><br><br><br><br><br><br><br><br>...<br><br><br><br><br><br><br><br><br><br>...<br><br><br><br><input autofocus>',
                '<meta charset="mac-farsi">¼script¾javascript:alert(379)¼/script¾',
                '<HTML xmlns:xss><?import namespace=(493)s" implementation="%(htc)s"><xss:xss>XSS</xss:xss></HTML>""","XML namespace."),("""<XML ID=(494)s"><I><B>&lt;IMG SRC="javas<!-- -->cript:javascript:alert(420)"&gt;</B></I></XML><SPAN DATASRC="#xss" DATAFLD="B" DATAFORMATAS="HTML"></SPAN>'
            ]



#print(Xnew)

In [24]:
Xnew = getVec(testXSS)
# make a prediction
#1 DecisionTreeClassifier
ynew1 = loaded_model1.predict(Xnew)
#2 SVC
ynew2 = loaded_model2.predict(Xnew)
#3 GaussianNB
ynew3 = loaded_model3.predict(Xnew)
#4 KNeighborsClassifier
ynew4 = loaded_model4.predict(Xnew)
#5 RandomForestClassifier
ynew5 = loaded_model5.predict(Xnew)
#6 MLPClassifier
ynew6 = loaded_model6.predict(Xnew)

Building the sample vector model...
*************************
Model Saved


##### Display the results 

In [25]:
# show the sample inputs and predicted outputs using a wighted value
xssCount = 0 
notXssCount = 0
for i in range(len(Xnew)):
    score = ((.175*ynew1[i])+(.15*ynew2[i])+(.05*ynew3[i])+(.075*ynew4[i])+(.25*ynew5[i])+(.3*ynew6[i]))
    print(ynew1[i])
    print(ynew2[i])
    print(ynew3[i])
    print(ynew4[i])
    print(ynew5[i])
    print(ynew6[i])
    print(score)
    if score >= .5:
        print("\033[1;31;1mXSS\033[0;0m => "+testXSS[i])
        xssCount += 1
    else:
        print("\033[1;32;1mNOT XSS\033[0;0m => "+testXSS[i])
        notXssCount += 1

print()
print("*------------- RESULTS -------------*")
print("\033[1;31;1mXSS\033[0;0m => "+str(xssCount))
print("\033[1;32;1mNOT XSS\033[0;0m => "+str(notXssCount))

1
1
1
0
1
1
0.925
XSS => <script>alert('xss')</script><script><script>
1
0
0
0
0
0
0.175
NOT XSS => hellomo
1
1
1
0
1
0
0.625
XSS => https://store.bentley.com/en/shop/search?term=%22%3E%3Cdetails%20open%20ontoggle=prompt(1337)%3ExxLouisLouisLouis
0
0
0
0
0
0
0.0
NOT XSS => ghfdhgdhjgd
1
1
1
0
1
1
0.925
XSS => uid%3D19%26list_page%3D%22%3E%3Cscript%3Ealert%28document.cookie%29%3B%3C/script%3E
0
0
0
0
0
0
0.0
NOT XSS => &template=en_search_error&postalCode=\';alert(0)//
1
1
1
1
1
1
1.0
XSS => &where=%3Cscript%3Ealert%28%27xss%27%29%3C%2Fscript%3E&loctypes=1003%2C1001%2C1000%2C1%2C9%2C5%2C11%2C13%2C19%2C20&from=hdr_localsearch
1
0
0
0
0
1
0.475
NOT XSS => http://mydata.com/sad/sd/qwd/qwde/qwe/?sessionid=12
0
0
0
0
0
0
0.0
NOT XSS => http://mydata.com?id=script
1
0
1
0
0
0
0.22499999999999998
NOT XSS => &';}},{scope:'email,user_about_me,user_hometown,user_interests,user_likes,user_status,user_website,user_birthday,publish_stream,publish_actions,offline_access'});}alert(0);b=function(respon